*This notebook descbites howo to select genes (features) of importance using Random Forests through the Boruta algorithm.*

**Background information**:
+ Random Forests
    + The [sklearn model used](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
    + [The excellent sklearn user guide](https://scikit-learn.org/stable/user_guide.html)
+ [Python Boruta algorithm](https://github.com/scikit-learn-contrib/boruta_py)

---

***Setting up the notebook***

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import holoviews as hv
hv.extension("bokeh")
%matplotlib inline

import GSForge as gsf

***Declaring used paths***

In [ ]:
# OS-independent path management.
from os import fspath, environ
from pathlib import Path

In [ ]:
OSF_PATH = Path(environ.get("GSFORGE_DEMO_DATA", default="~/GSForge_demo_data/osfstorage")).expanduser()
HYDRO_GEM_PATH = OSF_PATH.joinpath("AnnotatedGEMs", "oryza_sativa_hydro_raw.nc")
assert HYDRO_GEM_PATH.exists()

In [ ]:
OUTPUT_DIR = OSF_PATH.joinpath("Collections", "boruta")

***Loading a demo AnnotatedGEM***

In [ ]:
agem = gsf.AnnotatedGEM(HYDRO_GEM_PATH)
agem

In [ ]:
agem.data

## Finding Genes

Selecting relevent genes of importance (something that requires further experimentation to confirm) is not simple.
But we can make it appear so, by demonstrating the boruta feature selection method to construct a `GSForge.GeneSet`.

First we will need a model that, when trained, can rank or infer feature importance:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
selecting_model = RandomForestClassifier(
    class_weight='balanced',
    max_depth=4, 
    n_estimators=1000, 
    n_jobs=-1)

Now we can run the boruta feature selection method.

In [ ]:
%%time
boruta_result = gsf.operations.BorutaProspector(
    agem, 
    estimator=selecting_model, 
    annotation_variables="treatment",
    max_iter=2,
    perc=95)

This produces an `xarray.Dataset` object.

In [ ]:
boruta_result

### Creating a GeneSet from the result

Consider any metadata that should be stored, and add that to the `xarray.Dataset` result using `assign_attrs()`

```python
attrs = {"selection_model": str(selecting_model)}
boruta_result = boruta_result.assign_attrs(attrs)
```

You will need to convert nested dictionaries into json strings.

After the object is created you can easily see the number and percent of genes selected:

In [ ]:
boruta_treatment_gs = gsf.GeneSet(boruta_result, name="Boruta Treatment")
boruta_treatment_gs

## Using a `GeneSetCollection`

For running more than one boruta model in the notebook, we recommend creating them in a loop, and adding them to a collection, as demonstrated below.

Although if you want to run many different models, you should strongly consider using the [nextflow workflows](workflow_guide/workflow_guide.ipynb).

```python
from tqdm.notebook import tqdm

boruta_gsc = gsf.GeneSetCollection(gem=agem)

for target in tqdm(["treatment", "genotype"]):
    boruta_treatment_ds = gsf.operations.BorutaProspector(
        agem,
        estimator=selecting_model,
        annotation_variables=target,
        max_iter=50)
    
    boruta_gsc.gene_sets[target] = gsf.GeneSet(boruta_treatment_ds, name=f"Boruta_{target}")
    
```

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
boruta_gsc = gsf.GeneSetCollection(gem=agem)

for target in tqdm(["treatment", "genotype"]):
    boruta_treatment_ds = gsf.operations.BorutaProspector(
        agem,
        estimator=selecting_model,
        annotation_variables=target,
        perc=95,
        max_iter=50)
    
    boruta_gsc[target] = gsf.GeneSet(boruta_treatment_ds, name=f"Boruta_{target}")

In [ ]:
boruta_gsc

In [ ]:
OUTPUT_DIR

In [ ]:
boruta_gsc.save(OUTPUT_DIR)